# Create Ensembles for Projection tests
- Generates ensembles for projection models
- Ensemble size is 20
- 50 ensembles are created by sampling (without replacement) from the pool of models
- Save the ensemble predictions, uncertainties and statistics

## Imports

In [ ]:
import glob
import os
import pandas as pd

import initialise
import common
from model_utils import generate_ensembles

## Directories and other settings
- Update the model directories as required

In [ ]:
samples_file = os.path.join(common.DATASETS_DIR, 'samples_730days.csv')
model_dir = os.path.join(common.MODELS_DIR, 'evaluation_models')

precision = 3       # floating point precision for saved predictions
random_seed = 9876
overwrite = False  # Set to True to overwrite existing ensemble results

### Read Test Predictions
Read the prediction files for each test and retain the predictions that match the samples index. Either calculate the prediction statistics or read from the saved stats file (if available).

In [ ]:
def read_predictions(model_dir):
    model_predicts = []
    test_dirs = sorted(glob.glob(os.path.join(model_dir, f'test*')))
    for test_dir in test_dirs:
        test_predicts = []
        for run_dir in glob.glob(os.path.join(test_dir, 'run*')):
            preds_ = pd.read_csv(os.path.join(run_dir, 'predictions.csv'), index_col=0)
            test_predicts.append(preds_)
        model_predicts.append(test_predicts)
    return model_predicts, test_dirs

### Save Ensemble Predictions
Save the predictions made by the ensembles using the model of interest. Each size ensemble is stored in a separate CSV file. Columns are the individual ensemble predictions and rows are the samples.

In [ ]:
def write_ensembles(test_dirs, model_predicts, model_uncertainty, model_stats, precision):
    for num in range(len(model_predicts)):
        file_name = f"ensemble{common.ENSEMBLE_SIZE}_{common.ANALYSIS_MODEL}.csv"
        stats_fname = f"ensemble{common.ENSEMBLE_SIZE}_stats.csv"
        stds_fname = f"ensemble{common.ENSEMBLE_SIZE}_stds.csv"
        test_dir = os.path.join(model_dir, f'test{num}')
        test_dir = test_dirs[num]
        if overwrite or not os.path.exists(os.path.join(test_dir, file_name)):
            print(os.path.join(test_dir, file_name))
            print(os.path.join(test_dir, stats_fname))
            print(os.path.join(test_dir, stds_fname))
            df = pd.concat([pred_[common.ANALYSIS_MODEL] for pred_ in model_predicts[num]], axis=1, ignore_index=True).round(precision)
            df.to_csv(os.path.join(test_dir, file_name))
            df = pd.DataFrame([run.stack() for run in model_stats[num]]).T
            df.to_csv(os.path.join(test_dir, stats_fname))
            df = pd.concat([pred_[common.ANALYSIS_MODEL] for pred_ in uncertainty[num]], axis=1, ignore_index=True).round(precision)
            df.to_csv(os.path.join(test_dir, stds_fname))

### Generate the ensembles
Generate ensembles of various sizes for each of the models. For each ensemble size, randomly select the runs to ensemble, then create the ensembles. This is repeated for the desired number of ensembles of each size.

In [ ]:
model_predicts, test_dirs = read_predictions(model_dir)
predict, uncertainty, all_stats = generate_ensembles(model_predicts, common.ENSEMBLE_RUNS, common.ENSEMBLE_SIZE,
                                                     precision=precision, uncertainty=True, random_seed=random_seed)
write_ensembles(test_dirs, predict, uncertainty, all_stats, precision)